In [1]:
import pandas as pd
import yfinance as yf

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def get_data(ticker):
    # Define the URL to scrape
    url = f"https://finance.yahoo.com/quote/{ticker}/key-statistics/"

    # Define headers to simulate a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive'
    }

    # Make the request to the webpage
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract all tables
        tables = soup.find_all('table')
        
        # List to hold data for CSV
        csv_data = []

        # Process each table
        for i, table in enumerate(tables, start=1):
            # Extract table headers
            headers = [header.get_text(strip=True) for header in table.find_all('th')]
            
            # Extract table rows
            rows = table.find_all('tr')
            for row in rows:
                columns = [col.get_text(strip=True) for col in row.find_all('td')]
                # If the row contains data or headers, add it to the list
                if columns or (i == 1 and headers):
                    csv_data.append(columns)
                    
            # Add headers only if the table has them
            if headers and len(csv_data) > 0:
                csv_data.insert(0, headers)
        
        # Get the current date for the filename
        today = datetime.today().strftime('%Y-%m-%d')
        
        # Convert the data to a DataFrame and save to CSV
        df = pd.DataFrame(csv_data)
        df = df.transpose()
        df.columns = df.iloc[0]
        df = df[1:]
        #display(df)
        rm_cols = ['', None]
        df.drop(columns=rm_cols, inplace=True)
        df = df.iloc[[0]]

        df['Date'] = today
        df['Ticker'] = ticker
        df = df[['Date', 'Ticker'] + [col for col in df.columns if col not in ['Date', 'Ticker']]]
        df.set_index('Date', inplace=True)

        print(f"{ticker} saved")

        return df
    

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")


In [54]:
import time
tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].str.replace('.', '-').unique().tolist()
first50 = tickers[175:200]
final_df = pd.read_csv('data/new_data.csv', index_col='Date')
for ticker in first50:
    df = get_data(ticker)
    time.sleep(2)
    try:
        final_df = pd.concat([final_df, df])
    except:
        print(f'{ticker} was not added')
final_df.to_csv('data/new_data.csv')

EFX saved
EQIX saved
EQR saved
ESS saved
EL saved
ETSY saved
EG saved
EVRG saved
ES saved
EXC saved
EXPE saved
EXPD saved
EXR saved
XOM saved
FFIV saved
FDS saved
FICO saved
FAST saved
FRT saved
FDX saved
FIS saved
FITB saved
FSLR saved
FE saved
FI saved
FMC saved
F saved
FTNT saved
FTV saved
FOXA saved
FOX saved
BEN saved
FCX saved
GRMN saved
IT saved
GE saved
GEHC saved
GEV saved
GEN saved


KeyError: "['' None] not found in axis"